# mini sequences

In [ ]:
# macht mehrere sequences indem sie die funktion darunter called
def get_X_y

In [ ]:
# macht eine sequence
def get Xi_yi()

# Import libraries

In [1]:
# Data manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Deep Learning
import tensorflow as tf

In [2]:
from typing import Dict, List, Tuple, Sequence

# Import data

In [20]:
path = "/Users/niki/code/Niki827/watt_squad/eu_sidehustle/aggregating_preprocessing/data/04_preprocessed"

In [27]:
athens = pd.read_csv(f'{path}/Athens_preprocessed.csv')

In [28]:
berlin = pd.read_csv(f'{path}/Berlin_preprocessed.csv')

In [29]:
london = pd.read_csv(f'{path}/London_preprocessed.csv')

In [30]:
madrid = pd.read_csv(f'{path}/Madrid_preprocessed.csv')

In [31]:
paris = pd.read_csv(f'{path}/Paris_preprocessed.csv')

## Inspect data

In [32]:
paris.isna().sum()

,pv_output,direct_irradiance,diffuse_irradiance,reflected_irradiance,sun_height,temp,wind_speed,year_sin,year_cos
0,0.18438,0.005371,0.065585,0.031635,0.006396,0.365736,0.331266,1.721336e-02,0.999852
1,2.55676,0.332888,0.188187,0.137603,0.007762,0.318815,0.431817,3.442161e-02,0.999407
2,2.25707,0.287603,0.181857,0.127422,0.009273,0.290491,0.318518,5.161967e-02,0.998667
3,2.77577,0.364820,0.198938,0.151992,0.010888,0.251810,0.337021,6.880243e-02,0.997630
4,0.08309,0.000897,0.033462,0.015843,0.012606,0.356150,0.406667,8.596480e-02,0.996298
...,...,...,...,...,...,...,...,...,...
6934,2.03699,0.232890,0.227027,0.130902,0.001408,0.391161,0.478298,-6.880243e-02,0.997630
6935,1.26006,0.150560,0.133012,0.083684,0.002091,0.417086,0.548688,-5.161967e-02,0.998667
6936,0.43408,0.028270,0.108200,0.053192,0.002877,0.401442,0.559403,-3.442161e-02,0.999407
6937,0.28637,0.009971,0.092319,0.044569,0.003871,0.387687,0.454388,-1.721336e-02,0.999852


# Create suitable data structure

- separate train and test first (for instance 2/3 for train and 1/3 for test)
- then create sequences (2 years x and 1 year y)

## Separate train and test datasets
- separate train and test first (for instance 2/3 for train and 1/3 for test)
- add all train and test dataframes into one dictionary

In [103]:
def train_test_split(fold:pd.DataFrame) -> Tuple[pd.DataFrame]:
    # Calculate the index for two-thirds of the rows
    train_size = int(fold.shape[0] * (2 / 3))

    # Create a new DataFrame with the first two-thirds of rows
    fold_train = fold.iloc[:train_size]

    # Create a new DataFrame with the last third of rows
    fold_test = fold.iloc[train_size:]

    return (fold_train, fold_test)

## Create sequences (2 years x and 1 year y)

In [55]:
# Calculate max index to start a sequence from
train_number_sequences = dfs_train_test['paris_train'].shape[0] - 1095
test_number_sequences = dfs_train_test['paris_test'].shape[0] - 1095

### Define the function

In [67]:
def get_X_y_strides(fold: pd.DataFrame, input_length: int, output_length: int, sequence_stride: int):
    '''
    - slides through a `fold` Time Series (2D array) to create sequences of equal
        * `input_length` for X,
        * `output_length` for y,
    using a temporal gap `sequence_stride` between each sequence
    - returns a list of sequences, each as a 2D-array time series
    '''

    X, y = [], []

    for i in range(0, len(fold), sequence_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (i + input_length + output_length) >= len(fold):
            break
        X_i = fold.iloc[i:i + input_length, :]
        y_i = fold.iloc[i + input_length:i + input_length + output_length, :][['pv_output']] # index + length of sequence until index + length of seq. + length of target
        X.append(X_i)
        y.append(y_i)

    return np.array(X), np.array(y)

### Apply the function to our train folds

In [74]:
# Initialize dictionaries to store X_train and y_train for each city
X_train_dict = {}
y_train_dict = {}

# Iterate over each city in train_dfs
for city in train_dfs.keys():
    # Extract the training DataFrame for the current city
    train_df = train_dfs[city]
    
    # Call the get_X_y_strides function to generate X_train and y_train
    X_train, y_train = get_X_y_strides(train_df, input_length=730, output_length=365, sequence_stride=1)
    
    # Store the results in the dictionaries
    X_train_dict[city] = X_train
    y_train_dict[city] = y_train

# Now X_train_dict and y_train_dict contain the training datasets for each city

### Apply the function to our test folds

In [75]:
# Initialize dictionaries to store X_test and y_test for each city
X_test_dict = {}
y_test_dict = {}

# Iterate over each city in test_dfs
for city in test_dfs.keys():
    # Extract the test DataFrame for the current city
    test_df = test_dfs[city]
    
    # Call the get_X_y_strides function to generate X_test and y_test
    X_test, y_test = get_X_y_strides(test_df, input_length=730, output_length=365, sequence_stride=1)
    
    # Store the results in the dictionaries
    X_test_dict[city] = X_test
    y_test_dict[city] = y_test

# Now X_test_dict and y_test_dict contain the testing datasets for each city

# (2.1) 💻 A Recurrent Neural Network: the `LSTM`

## Init model

In [81]:
def init_model(X_train, y_train):
    # 1 - RNN architecture
    model = models.Sequential()
    ## 1.1 - Recurrent Layer
    model.add(layers.LSTM(64, 
                          activation='tanh', 
                          return_sequences=False,
                          kernel_regularizer=L1L2(l1=0.05, l2=0.05),
                          input_shape=(X_train.shape[1], X_train.shape[2])  # Specify input shape
                          ))
    ## 1.2 - Predictive Dense Layers
    output_length = y_train.shape[1]
    model.add(layers.Dense(output_length, activation='linear'))  

    # 2 - Compiler
    adam = optimizers.Adam(learning_rate=0.02)    
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])
    
    return model

In [82]:
model = init_model(X_train_dict['paris_train'], y_train_dict['paris_train'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                18944     
                                                                 
 dense_1 (Dense)             (None, 365)               23725     
                                                                 
Total params: 42,669
Trainable params: 42,669
Non-trainable params: 0
_________________________________________________________________


## Plot History

In [83]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # --- LOSS: MSE --- 
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- METRICS:MAE ---
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax

## Fit model

In [84]:
# YOUR CODE HERE
from tensorflow.keras.callbacks import EarlyStopping

def fit_model(model: tf.keras.Model, verbose=1) -> Tuple[tf.keras.Model, dict]:

    es = EarlyStopping(monitor = "val_loss",
                      patience = 3,
                      mode = "min",
                      restore_best_weights = True)


    history = model.fit(X_train_dict['paris_train'], y_train_dict['paris_train'],
                        validation_split = 0.3,
                        shuffle = False,     # the order matters!!!
                        batch_size = 32,
                        epochs = 50,
                        callbacks = [es],
                        verbose = verbose)

    return model, history   # returns a tuple: (model, dict(history))

## Simple LSTM without Cross Val

In [86]:
# 1 - Initialising the RNN model
# ====================================

model = init_model(X_train_dict['paris_train'], y_train_dict['paris_train'])
model.summary()

# 2 - Training
# ====================================
model, history = fit_model(model)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                18944     
                                                                 
 dense_2 (Dense)             (None, 365)               23725     
                                                                 
Total params: 42,669
Trainable params: 42,669
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


2024-12-02 11:59:36.940622: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


78/78 [==============================] - 11s 134ms/step - loss: 4.9038 - mae: 1.6630 - val_loss: 3.3683 - val_mae: 1.5026
Epoch 2/50
78/78 [==============================] - 10s 131ms/step - loss: 3.6041 - mae: 1.5599 - val_loss: 3.3435 - val_mae: 1.4992
Epoch 3/50
78/78 [==============================] - 10s 132ms/step - loss: 3.4891 - mae: 1.5347 - val_loss: 3.3221 - val_mae: 1.5000
Epoch 4/50
78/78 [==============================] - 10s 129ms/step - loss: 3.4514 - mae: 1.5236 - val_loss: 3.3301 - val_mae: 1.4942
Epoch 5/50
78/78 [==============================] - 10s 130ms/step - loss: 3.4044 - mae: 1.5152 - val_loss: 3.2991 - val_mae: 1.4937
Epoch 6/50
78/78 [==============================] - 10s 129ms/step - loss: 3.3857 - mae: 1.5106 - val_loss: 3.3144 - val_mae: 1.4937
Epoch 7/50
78/78 [==============================] - 10s 129ms/step - loss: 3.3734 - mae: 1.5074 - val_loss: 3.3140 - val_mae: 1.4934
Epoch 8/50
78/78 [==============================] - 10s 129ms/step - loss: 3.364

# Create a benchmark model

In [98]:
from tensorflow.keras.layers import Lambda  # Wraps arbitrary expressions as a Layer object

def init_baseline():

    # YOUR CODE HERE
    model = models.Sequential() 
    # a layer to take the last value of the sequence and output it
    model.add(layers.Lambda(lambda x: x[:,-1,1,None]))  # all sequences, last day, 1 feature (temperature)
    

    adam = optimizers.Adam(learning_rate=0.02)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])

    return model

In [99]:
baseline_model = init_baseline()
baseline_score = baseline_model.evaluate(X_test_dict['paris_test'], y_test_dict['paris_test'])
print(f"- The Baseline MAE on the test set is equal to {round(baseline_score[1],2)} kwH.")

39/39 [==============================] - 0s 445us/step - loss: 11.2137 - mae: 2.8608
- The Baseline MAE on the test set is equal to 2.86 kwH.


In [100]:
print(f"- The LSTM MAE on the test set is equal to {round(res[1],2)} Celsius degrees")
print(f"🔥 Improvement of the LSTM model over the baseline (on this fold for the test set) = : {round((1 - (res[1]/baseline_score[1]))*100,2)} % 🔥")

NameError: name 'res' is not defined

## Cross Val LSTM

In [105]:
from tensorflow.keras.callbacks import EarlyStopping

def cross_validate_baseline_and_lstm():
    '''
    This function cross-validates 
    - the "last seen value" baseline model
    - the RNN model
    '''

    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []

    # 0 - Creating folds
    # =========================================  
    # Assume you have a dictionary with DataFrames for each city
    city_dfs = {
        'paris': paris,
        'athens': athens,
        'berlin': berlin,
        'london': london,
        'madrid': madrid
    }
    # Create a list of DataFrames (folds) for each city
    folds = [city_dfs[city] for city in city_dfs.keys()]

    for fold_id, fold in enumerate(folds):
        # 1 - Train/Test split the current fold
        # =========================================
        (fold_train, fold_test) = train_test_split(fold) # function we coded to split train/test                                 

        X_train, y_train = get_X_y_strides(fold_train, input_length=730, output_length=365, sequence_stride=1)  # function we coded to get multiple
        X_test, y_test = get_X_y_strides(fold_test, input_length=730, output_length=365, sequence_stride=1)       # sequences from a fold

        # 2 - Modelling
        # =========================================
        
        ##### Baseline Model
        baseline_model = init_baseline()
        mae_baseline = baseline_model.evaluate(X_test, y_test, verbose=0)[1]   # evaluating baseline model (metric)
        list_of_mae_baseline_model.append(mae_baseline)
        print("-"*50)
        print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")

        ##### LSTM Model
        model = init_model(X_train, y_train)
        es = EarlyStopping(monitor = 'val_mae',
                          mode = "min",
                           patience = 2, 
                           restore_best_weights = True)
        history = model.fit(X_train, y_train,
                           validation_split = 0.3,
                           shuffle = False,
                           batch_size = 32,
                           epochs = 50,
                           callbacks = [es],
                           verbose = 0)
        res = model.evaluate(X_test, y_test, verbose = 0) # evaluating LSTM (metric)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)
        print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")
        
        ##### Comparison LSTM vs Baseline for the current fold
        print(f"🏋🏽‍♂️ improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")

    return list_of_mae_baseline_model, list_of_mae_recurrent_model

In [106]:
%%time
# WARNING : it takes 75 minutes to run this cell 
mae_baselines, mae_lstms = cross_validate_baseline_and_lstm()

--------------------------------------------------
MAE baseline fold n°0 = 2.86
MAE LSTM fold n°0 = 1.52
🏋🏽‍♂️ improvement over baseline: 46.93 % 

--------------------------------------------------
MAE baseline fold n°1 = 3.27
MAE LSTM fold n°1 = 1.4
🏋🏽‍♂️ improvement over baseline: 57.2 % 

--------------------------------------------------
MAE baseline fold n°2 = 2.56
MAE LSTM fold n°2 = 1.56
🏋🏽‍♂️ improvement over baseline: 39.09 % 

--------------------------------------------------
MAE baseline fold n°3 = 2.64
MAE LSTM fold n°3 = 1.43
🏋🏽‍♂️ improvement over baseline: 45.85 % 

--------------------------------------------------
MAE baseline fold n°4 = 3.79
MAE LSTM fold n°4 = 1.2
🏋🏽‍♂️ improvement over baseline: 68.41 % 

CPU times: user 4min 35s, sys: 17 s, total: 4min 52s
Wall time: 4min 48s


# Create the final model

In [ ]:
# --- SEQUENCE A (Paris)

day_1 = [10, 25, 60]  # OBSERVATION 1 [temperature, speed, pollution]
day_2 = [13, 10, 80]  # OBSERVATION 2 [temperature, speed, pollution]
day_3 = [ 9,  5, 90]  # OBSERVATION 3 [temperature, speed, pollution]
day_4 = [ 7,  0, 100]  # OBSERVATION 4 [temperature, speed, pollution]

sequence_a = [day_1, day_2, day_3, day_4]

y_a = [110] # Pollution at day 5

# --- SEQUENCE B (Berlin)
sequence_b = [[25, 20, 30], [26, 24, 50], [28, 20, 80], [22, 3, 110]]
y_b = [125]

# --- SEQUENCE C (London)
sequence_c = [[15, 10, 60], [25, 20, 65], [35, 10, 75], [36, 15, 70]]
y_c = [75]

X = np.array([sequence_a, sequence_b, sequence_c]).astype(np.float32)
y = np.expand_dims(np.array([y_a, y_b, y_c]).astype(np.float32), axis=-1)

print(X.shape)
print(y.shape)

In [ ]:
model_2 = Sequential()
model_2.add(layers.SimpleRNN(2, return_sequences=True))
model_2.add(layers.Dense(1, activation='relu'))
model_2.compile(loss='mse', optimizer='adam')
model_2.fit(X, y, verbose=0)

In [ ]:
##### LSTM Model
model = init_model(X_train, y_train)
es = EarlyStopping(monitor = 'val_mae',
                  mode = "min",
                   patience = 2, 
                   restore_best_weights = True)
history = model.fit(X_train, y_train,
                   validation_split = 0.3,
                   shuffle = False,
                   batch_size = 32,
                   epochs = 50,
                   callbacks = [es],
                   verbose = 0)
res = model.evaluate(X_test, y_test, verbose = 0) # evaluating LSTM (metric)
mae_lstm = res[1]
list_of_mae_recurrent_model.append(mae_lstm)
print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")